In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import ftfy
import textacy
import csv
from gensim.utils import simple_preprocess
import unicodedata
import swifter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import regex as re

/home/ubuntu/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


#### Preprocessing

Lots of stuff here is embedding dependant. For instance, you should filter stop words when using word2vec, but you should not do that for glove. Glove also uses specific preprocessing procedure for twitter. Word2vec recommends stripping most punctuation. FastText replaces things like New York with new_york.

In [20]:
PATH_TO_DATASETS=Path('../../hedwig-data/datasets')
STOP_WORDS = list(set(stopwords.words('english')))
FLAGS = re.MULTILINE | re.DOTALL
ALPHABET = dict(map(lambda t: (t[1], t[0]),
                    enumerate(list("abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=<>()[]{}"))))

def get_stop_list(text_list, text_col=1, min_count=10):
    word_freq = {}
    remove = {}
    for text in text_list:
        for word in text.split():
            if word in word_freq.keys():
                word_freq[word] += 1
            else:
                word_freq[word] = 1
    for k, v in word_freq.items():
        if v < 3:
            remove[k]=True

    return remove

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = " {} ".format(hashtag_body.lower())
    else:
        result = " ".join([""] + [re.sub(r"([A-Z])",r" \1", hashtag_body, flags=FLAGS)])
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " "

def tweet_preprocessing(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = text.replace("@ HASHTAG ", "#")
    text = text.replace("@ USER ", "<user> ")
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = text.replace("@ URL", "<url>")
    text = text.replace(" [ URL ] ", "<url>")
    text = text.replace(" [ USER ] ", "<number>")
    text = text.replace(" [ NUMBER ] ", "<user>")
    text = text.replace(" [ HASHTAG ] ", "<hashtag>")
    text = re_sub(r"([A-Z]){2,}", allcaps)
    
    return text.lower()
                 
def is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def clean_text(text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or is_control(char):
            continue
        if is_whitespace(char):
            output.append(" ")
        else:
            output.append(char)
    return "".join(output)

def fix_contractions(text):
    text = re.sub(
        r"(\b)([Aa]re|[Cc]ould|[Dd]id|[Dd]oes|[Dd]o|[Hh]ad|[Hh]as|[Hh]ave|[Ii]s|[Mm]ight|[Mm]ust|[Ss]hould|[Ww]ere|[Ww]ould) n't",
        r"\1\2 not",
        text,
    )
    text = re.sub(
        r"(\b)([Hh]e|[Ii]|[Ss]he|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou) 'll",
        r"\1\2 will",
        text,
    )
    text = re.sub(r"(\b)([Tt]here|[Hh]ere) 's", r"\1\2 is", text)
    text = re.sub(r"(\b)([Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou) 're", r"\1\2 are", text)
    text = re.sub(
        r"(\b)([Ii]|[Ss]hould|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Ww]ould|[Yy]ou) 've",
        r"\1\2 have",
        text,
    )
    text = re.sub(
        r"(\b)([Hh]e|[Ii]|[Ss]he|[Tt]hey|[Ww]e|[Yy]ou) 'd",
        r"\1\2 would",
        text,
    )
    # non-standard
    text = re.sub(r"(\b)([Cc]a) n't", r"\1\2n not", text)
    text = re.sub(r"(\b)([Ii]) 'm", r"\1\2 am", text)
    text = re.sub(r"(\b)([Ll]et) 's", r"\1\2 us", text)
    text = re.sub(r"(\b)([Ww]) on't", r"\1\2ill not", text)
    text = re.sub(r"(\b)([Ss]) han't", r"\1\2hall not", text)
    text = re.sub(r"(\b)([Yy])(?: 'all|a 'll)", r"\1\2ou all", text)
    #####################################################
    text = re.sub(
        r"(\b)([Aa]re|[Cc]ould|[Dd]id|[Dd]oes|[Dd]o|[Hh]ad|[Hh]as|[Hh]ave|[Ii]s|[Mm]ight|[Mm]ust|[Ss]hould|[Ww]ere|[Ww]ould) n ' t",
        r"\1\2 not ",
        text,
    )
    text = re.sub(
        r"(\b)([Hh]e|[Ii]|[Ss]he|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou) ' ll ",
        r"\1\2 will ",
        text,
    )
    text = re.sub(r"(\b)([Tt]here|[Hh]ere) ' s ", r"\1\2 is", text)
    text = re.sub(r"(\b)([Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Yy]ou) ' re ", r"\1\2 are", text)
    text = re.sub(
        r"(\b)([Ii]|[Ss]hould|[Tt]hey|[Ww]e|[Ww]hat|[Ww]ho|[Ww]ould|[Yy]ou) ' ve ",
        r"\1\2 have ",
        text,
    )

    text = re.sub(
        r"(\b)([Hh]e|[Ii]|[Ss]he|[Tt]hey|[Ww]e|[Yy]ou) ' d ",
        r"\1\2 would ",
        text,
    )
    # non-standard
    text = re.sub(r"(\b)([Cc]a) n ' t ", r"\1\2n not ", text)
    text = re.sub(r"(\b)([Ii]) ' m ", r"\1\2 am ", text)
    text = re.sub(r"(\b)([Ll]et) ' s ", r"\1\2 us ", text)
    text = re.sub(r"(\b)([Ww]) on ' t ", r"\1\2ill not ", text)
    text = re.sub(r"(\b)([Ss]ha) n ' t ", r"\1\2ll not ", text)
    text = re.sub(r"(\b)([Yy])(?: ' all | a ' ll )", r"\1\2ou all ", text)
    text=text.replace(" 's ", "'s ").replace(" ' s ", "'s ").replace(" i ' m ", " i am ")
    return text
    
def soft_preprocess(df):
    df.iloc[:,1]=df.iloc[:,1].swifter.apply(ftfy.fix_text)
    df.iloc[:,1]=df.iloc[:,1].swifter.apply(clean_text)
    df.iloc[:,1]=df.iloc[:,1].swifter.apply(
        lambda x: x.replace(
            '"', "").replace("\n", " ").replace("\\","").replace("`","'").replace("& amp ;", " and "))
    df.iloc[:,1]=df.iloc[:,1].swifter.apply(fix_contractions)
    return df

def df_to_hedwig_tsv(df, dsname, outfilename, num_labels_in_col, preprocess, is_twitter_process=True, stop_words=[],
                     label_cols=[0], text_col=1):
    def to_tsv(outfpath, labels, texts):
        with open(outfpath, 'w', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t')
            for label, text in zip(labels, texts):
                if len(text.strip()) > 10:
                    writer.writerow([label, text])
    df = preprocess(df)
    if is_twitter_process:
        df.iloc[:,1]=df.iloc[:,1].swifter.apply(tweet_preprocessing)
    df.iloc[:,1] = df.iloc[:,1].swifter.apply(lambda text: " ".join(
        [word for word in text.split() if word not in stop_words and len(word) < 15]).strip())
    toremove=get_stop_list(df.iloc[:,1].tolist())
    df.iloc[:,1] = df.iloc[:,1].swifter.apply(lambda text: " ".join(
        [word for word in text.split() if word not in toremove]).strip())
    df.iloc[:,0] = df.swifter.apply(lambda row: ''.join([str(lbl) for lbl in row[label_cols]]), axis=1)
    df = df.iloc[:,[0, 1]]
    df.iloc[:,0]=df.iloc[:,0].astype('str')
    df.iloc[:,0]=df.iloc[:,0].swifter.apply(
        lambda x: x if len(x) == num_labels_in_col else ''.join(
            ['0' for i in range(num_labels_in_col-len(x))]
        )+str(x)
    )
    dspath=PATH_TO_DATASETS/dsname
    outfpath = dspath/outfilename
    df = df.sample(frac=1.0)
    to_tsv(outfpath, df.iloc[:,0].tolist(), df.iloc[:,1].tolist())
    return df


In [21]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/dev.csv')


In [22]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/dev.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='dev.tsv', num_labels_in_col=4,
                     preprocess=soft_preprocess, is_twitter_process=True)
tdf

,label,text
26199,0000,"user way , briefly legend canterbury telling o..."
3296,1000,. thx much ! ! best part ( spotting new trends...
21525,1011,"still hurts . first page expect get better , ...."
43665,0100,"believe sa bunso pa namin . <user> sometimes ,..."
4119,1000,<user> . <url> / happiest little bookmark ! ##...
24612,1101,slowly becoming one fangirls wants draw time m...
26095,1111,"/ favorite things <user> <user> . hey , help u..."
39200,0001,heard one best movies last years <url> / revie...
29330,0000,", verb <url> / <user> etsy highlight / / owl s..."
41960,0001,"sauna . shower , cold water constantly always ..."


In [23]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/test.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='test.tsv', num_labels_in_col=4,
                     preprocess=soft_preprocess, is_twitter_process=True)


In [24]:
df = pd.read_csv('../../hedwig-data/datasets/MBTI/train.csv')
tdf=df_to_hedwig_tsv(df, dsname="MBTI", outfilename='train.tsv', num_labels_in_col=4,
                     preprocess=soft_preprocess, is_twitter_process=True)


KeyboardInterrupt: 

In [ ]:
tdf